# 参数管理
## 访问参数

In [3]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4,8),nn.ReLU(),nn.Linear(8,1))
X = torch.rand(size=(2,4))
net(X)

tensor([[0.2360],
        [0.0817]], grad_fn=<AddmmBackward0>)

In [9]:
# 访问第二个全连接层的参数
print(net[2].state_dict()) 

OrderedDict([('weight', tensor([[ 0.1170, -0.0517,  0.1846, -0.0530,  0.1166, -0.1339,  0.1509, -0.0237]])), ('bias', tensor([0.0247]))])


In [10]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.0247], requires_grad=True)
tensor([0.0247])


In [11]:
# 除了参数的值，还能访问参数的梯度
net[2].weight.grad == None

True

我们也可以通过递归来提取每个子块的参数,下面是访问第一个全连接层的参数和访问所有层

In [15]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [16]:
net.state_dict()['2.bias'].data

tensor([0.0247])

## 参数初始化
### 内置初始化

In [18]:
# 初始化为标准差为0.01的高斯随机变量，偏置参数设置为0
def init_normal(m):
  if type(m) == nn.Linear:
    nn.init.normal_(m.weight, mean=0, std=0.01)
    nn.init.zeros_(m.bias)
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0023,  0.0108, -0.0014, -0.0250]), tensor(0.))

In [20]:
# 初始化为固定常数
def init_constant(m):
  if type(m) == nn.Linear:
    nn.init.constant_(m.weight, 1)
    nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [26]:
# 对不同层使用不同的初始化方法
net[0].apply(init_normal)
net[2].apply(init_constant)
print(net[0].weight.data, net[0].bias.data)
print(net[2].weight.data, net[2].bias.data)

tensor([[ 1.4858e-03, -9.0825e-03,  6.5241e-03,  1.0030e-02],
        [ 1.6660e-02,  2.2475e-03, -5.8466e-03, -1.1756e-03],
        [ 5.6816e-03, -1.5601e-02, -8.7184e-03, -3.8477e-03],
        [ 3.7585e-03,  8.0591e-04,  1.4646e-02,  2.3250e-02],
        [-5.8566e-05,  1.6466e-03,  1.7142e-02,  1.4015e-02],
        [-1.7782e-02, -4.6041e-03, -9.8881e-03,  3.1566e-04],
        [ 1.0631e-02,  1.7593e-02, -9.8710e-03, -9.0668e-03],
        [-7.1681e-03,  1.3963e-02, -1.0594e-06, -7.6700e-04]]) tensor([0., 0., 0., 0., 0., 0., 0., 0.])
tensor([[1., 1., 1., 1., 1., 1., 1., 1.]]) tensor([0.])


### 自定义初始化

### 参数绑定
在多个层之间共享参数，定义一个稠密层.
**由于模型参数包含梯度，在反向传播时，第二个隐藏层和第三个隐藏层的梯度会加在一起**。

In [28]:
# 给共享层一个名称，以便可以引用它的参数
shared = nn.Linear(8,8)
net = nn.Sequential(nn.Linear(4,8),nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8,1))
net(X)
# 检测参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0,0] = 100
# 确保它们实际上是同一个对象，而不只是相同的值
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])
